In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

In [4]:
import torch

from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
from optimizers_impl.sam.sam import SAM
from utility.bypass_bn import enable_running_stats, disable_running_stats


In [5]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        optimizer.zero_grad()

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions.logits, targets, smoothing=0.0001)
        loss.mean().backward()

        optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.logits, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler(epoch)

In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.0001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [7]:
if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    # model_name_path =''
    model_name = 'inceptionv3-sgd'
    initialize(seed=111)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
    model.num_classes = 2
    model.eval()

    dataset = Coronary(batch_size=10, threads=2, img_size_definition=299)
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.1, momentum=0.9)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.0, weight_decay=0.7)
    scheduler = StepLR(optimizer, 0.0004, 50)
#     scheduler = StepLR(optimizer, 0.0002, 50)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


100
300
880
┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      4.4484  │      0.28 %  ┃   4.000e-04  │       00:09  ┃      3.2220  │     48.67 %  ┃
┃           1  ┃      1.9083  │      0.76 %  ┃   4.000e-04  │       00:05  ┃      1.6812  │     75.33 %  ┃
┃           2  ┃      0.9745  │      0.80 %  ┃   4.000e-04  │       00:05  ┃      1.1073  │     80.67 %  ┃
┃           3  ┃      0.7255  │      0.84 %  ┃   4.000e-04  │       00:05  ┃      0.9037  │     81.00 %  ┃
┃           4  ┃      0.6173  │      0.86 %  ┃   4.000e-04  │       00:05  ┃      0.7654  │     83.33 %  ┃
┃           5  ┃      0.5647  │      0.87 %  ┃   4.000e-04  │       00:05  ┃      0.7719  │     83.67 %  ┃
┃           6  ┃      0.5550  │      0.87 %  ┃   4.000e-04  │       00:05  ┃      0.7353  │     82.67 %  ┃
┃           7  ┃      0.5460  │      0.87 %  ┃   4.000e-04  │       00:05  ┃      0.7768  │     82.00 %  ┃
┃           8  ┃      0.5513  │      0.87 %  ┃   4.000e-04  │       00:05  ┃      0.7331  │     85.33 %  ┃
┃           9  ┃      0.5341  │      

In [8]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [9]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


tensor(0.9901, device='cuda:0')
tensor(0.9900, device='cuda:0')
tensor(0.9804, device='cuda:0')
tensor(0.9800, device='cuda:0')
tensor(1., device='cuda:0')
